<a href="https://colab.research.google.com/github/FadelYang/craft-it-object-detection-model/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#importing required Libraries
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pandas as pd
import sys
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Preparing Dataset

## Connect To Kaggle And Download Datasets

this code is used to connect Google Colab and Kaggle, run it just once for download the datasets from kaggle, uncomment it first if you want to run these cell below

In [8]:
# # copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
# !mkdir /.kaggle
# !mv kaggle.json /.kaggle
# !mv /.kaggle /root/
# !chmod 600 ~/.kaggle/kaggle.json

In [9]:
# # Download datasets from Kaggle.
# !kaggle datasets download -d kshitizgajurel042/recyclable-images-with-annotations-for-detection

In [10]:
# !unzip recyclable-images-with-annotations-for-detection.zip

In [11]:
# !mv 'Final Data' recyclable-images-datasets

## Inspect Dataset

In [12]:
# Define function for read annotations
def read_annotations(annotation_file):
  with open(annotation_file, 'r') as file:
    lines = file.readlines()
    annotations = []
    for line in lines:
      parts = line.strip().split()
      class_label = parts[0]
      x_min, y_min, x_max, y_max = map(float, parts[1:])
      annotations.append({
          'class_label': class_label,
          'x_min': x_min,
          'y_min': y_min,
          'x_max': x_max,
          'y_max': y_max
      })
    return annotations

In [13]:
# Preprocessing data
data_dir = os.path.join(os.getcwd(), 'recyclable-images-datasets')

def preprocess_data(data_dir):
  images = []
  annotations = []

  image_folder = os.path.join(data_dir, 'images', 'train')
  label_folder = os.path.join(data_dir, 'labels', 'train')

  for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
      image_path = os.path.join(image_folder, image_filename)
      annotation_filename = image_filename.replace('.jpg', '.txt').replace('.png', '.txt')
      annotation_path = os.path.join(label_folder, annotation_filename)

      image = cv2.imread(image_path)

      # Resize image.
      image = cv2.resize(image, (416, 416))

      # Normalize image
      image = image/255.0

      if os.path.exists(annotation_path):
        annotation = read_annotations(annotation_path)
        images.append(image)
        annotations.append(annotation)
      else:
        print(f'Annotation file for image {image_filename} not found')

  return images, annotations

In [14]:
images, annotations = preprocess_data(data_dir)